In [104]:
#!pip install anvil-uplink


In [105]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install "azure-identity>=1.15.0"
! pip install pandas python-docx
!pip install PyDrive

import os
from openai import AzureOpenAI
#from google.colab import drive
#drive.mount('/content/drive')


#Import the Data Source

In [106]:
import pandas as pd
dataDict = pd.DataFrame(columns=['dataSource', 'dataString'])
dataDict

,dataSource,dataString


In [107]:
ClassSchedule_path = "https://docs.google.com/spreadsheets/d/1R3ptCqJdlHhW4ZP4KElZ1zdDmhRKs09s/export"
df = pd.read_excel(ClassSchedule_path)
for i in df:
  df["combined"] = "Week: " + df['Week'] + "; Section: " + df['Section'] + "; Date: " + df.Date.astype(str)+ "; Time: " + df.Time.astype(str) + "; Class: " + df.Class
temp = {'dataSource':'Class Schedule',
            'dataString':df["combined"].to_string(index = False)}
dataDict=dataDict._append(temp, ignore_index = True)
dataDict

instructorList_path = "https://docs.google.com/spreadsheets/d/1WQoDZ6S_xfbLra-qfwGswiLZa5bxqR-1/export"
df = pd.read_excel(instructorList_path)
for i in df:
  df["combined"] = "Section: " + df['Section'] + "; Instructor: " + df['Instructor'] + "; Email: " + df['email']
temp = {'dataSource':'Instructor List',
            'dataString':df["combined"].to_string(index = False)}
dataDict=dataDict._append(temp, ignore_index = True)
dataDict


,dataSource,dataString
0,Class Schedule,Week: Week 1; Section: all sections; Date: 202...
1,Instructor List,Section: T01 ; Instructor: Miss Karina Yuen; E...


In [109]:
#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex link : https://drive.google.com/file/d/1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp/view?usp=sharing
# Get the id from the link 1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp
downloaded = drive.CreateFile({'id':"1Cinn1LBaU3g6e1uGXA6xHQW6gN5Sj6gT"})
downloaded.GetContentFile('Assessment.docx')

downloaded = drive.CreateFile({'id':"1om4jb47oviqNZJ-YapEBuAU97c51CK_1"})
downloaded.GetContentFile('Admin FAQ.docx')
https://docs.google.com/document/d//edit?usp=drive_link&ouid=101099163109032005031&rtpof=true&sd=true

In [110]:
from docx import Document

doc = Document('Assessment.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
#print('\n'.join(full_text))

temp = {'dataSource':'Assessment',
            'dataString':'\n'.join(full_text)}
dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Admin FAQ.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
#print('\n'.join(full_text))

temp = {'dataSource':'Admin FAQ',
            'dataString':'\n'.join(full_text)}
dataDict=dataDict._append(temp, ignore_index = True)

dataDict

,dataSource,dataString
0,Class Schedule,Week: Week 1; Section: all sections; Date: 202...
1,Instructor List,Section: T01 ; Instructor: Miss Karina Yuen; E...
2,Assessment,Grading:\nThe course is graded on Pass/Fail.\n...


#Call AzureOpenAI API

In [111]:
#dotenv.load_dotenv('drive/MyDrive/Colab Notebooks/chatbot/chatbot.env')
#OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [112]:
from google.colab import userdata
# Gateway setting to OpenAI API
client = AzureOpenAI(
    # This is the default and can be omitted
    #azure_endpoint = os.environ.get('ENDPOINT'),
    #api_key = os.environ.get('OPENAI_API_KEY'),
    azure_endpoint = userdata.get('ENDPOINT'),
    api_key = userdata.get('OPENAI_API_KEY'),
    api_version="2024-05-01-preview",
)
fixed_tokens = 0
variable_tokens = 0
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))

Fix Cost (Tokens): 0
Variable Cost (Tokens): 0


## Embed Data Source

In [113]:
def generate_embeddings(text,fix_variable):
  global fixed_tokens
  global variable_tokens
  embedResult = client.embeddings.create(input = text, model="text-embedding-ada-002")
  #print(embedResult.usage.total_tokens)
  if fix_variable == 0:
    fixed_tokens+=(embedResult.usage.total_tokens)
  else:
    variable_tokens+=(embedResult.usage.total_tokens)
  return embedResult.data[0].embedding

dataDict['embeding'] = dataDict["dataString"].apply(lambda x : generate_embeddings(x,0))

print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
dataDict

Fix Cost (Tokens): 2054
Variable Cost (Tokens): 0


,dataSource,dataString,embeding
0,Class Schedule,Week: Week 1; Section: all sections; Date: 202...,"[-0.015611161477863789, -0.009069669991731644,..."
1,Instructor List,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.018751846626400948, -0.021726373583078384,..."
2,Assessment,Grading:\nThe course is graded on Pass/Fail.\n...,"[0.002250620163977146, 7.337156421272084e-05, ..."


## UI connection

In [114]:
#import anvil.server
#anvil.server.connect("server_SIEUKZS6SEZOEB42G6HWUISU-YKWKNMBS3233ZXGH")

#for anvil web interface
#@anvil.server.callable
def anvil_get_response(message):
    dataString=df.to_string(index = True) #transfer data into string
    chat_completion = client.chat.completions.create(
        model = 'gpt-35-turbo-16k',
        temperature = 0,
        max_tokens = 50,
        messages = [
            {"role": "system", "content": "You are a teaching assistant in PDEV6800Z. Answer student's questions regarding PDEV6800Z."},
            {"role": "user", "content": message},
            {"role": "assistant", "content": dataString}
        ]
    )

    # print token usage
    print(chat_completion.to_json())
    response = chat_completion.choices[0].message.content
    # return the response
    return response

## Handling Query

In [115]:
#basic query
# Updated context and initial prompt for the chatbot
initial_prompt = """
"Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.

Which session are you in (T01-T07)?"
"""

context = """
Role:
You are the Secretary, an AI chatbot designed to assist with administrative matters for PDEV6800Z.

Functions:
- Answer questions about scheduling, registration, policy, and course logistics.
- Use the syllabus and session schedules (T01-T07) for accurate responses.

Interaction Guidelines:
- Address users as "you" for engagement.
- Provide clear, concise, and actionable answers.
- Use markdown to structure complex responses.
- Ask for the student's session (T01-T07) and confirm it before assisting.

Rules for Engagement:
- Only address administrative issues. Redirect other questions to the instructor.
- Maintain a friendly and professional tone.
- Use indexed summaries and schedules for responses.
- Ask for additional information if needed.
- Acknowledge unanswered questions and suggest contacting the instructor.
- Ask if there is anything else they need help with at the end of each interaction.

You must ask the following prompt to the user as the begining:
"""+initial_prompt
# added this as well

def get_response(message, dataSource):
    global variable_tokens
    mask = dataDict['dataSource'].values == dataSource
    target_file = dataDict.loc[mask]
    if len(target_file)>0:
      dataString = target_file['dataString'].iloc[0]
    else:
      dataString=""
    assistantContent = """Use the below text to answer user's question. If the answer cannot be found, write 'I don't know.'"

            Text:
            \"\"\"
            {dataString}
            \"\"\"
            """
    chat_completion = client.chat.completions.create(
      model = 'gpt-35-turbo-16k',
      temperature = 0,
      messages = [
            {"role": "system", "content": context}, #changed this~~~~
            #{"role": "user", "content": initial_prompt}, #changed this~~~~
            {"role": "user", "content": message},
            {"role": "assistant", "content": assistantContent}
      ]
    )

    #print(assistantContent)
    variable_tokens+=chat_completion.usage.completion_tokens
    response = chat_completion.choices[0].message.content
    # return the response
    return response

In [116]:
#test for basic query
response = get_response("What is the First Class about?" , "Assessment" )

In [117]:
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
response

Fix Cost (Tokens): 2054
Variable Cost (Tokens): 66


"Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.\n\nWhich session are you in (T01-T07)?"

In [118]:
#find the most relevant documents
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_docs(user_query, top_n=1):
  query_embedding = generate_embeddings(user_query,1)
  dataDict["similarities"] = dataDict.embeding.apply(lambda x: cosine_similarity(x, query_embedding))

  res = (
    dataDict.sort_values("similarities", ascending=False).head(top_n)
  )
  return res


In [119]:
#test for searching relevant files
res = search_docs("When is the first class for T03?")

In [120]:
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
print(res.dataSource.iloc[0])
res.head()

Fix Cost (Tokens): 2054
Variable Cost (Tokens): 75
Instructor List


,dataSource,dataString,embeding,similarities
1,Instructor List,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.018751846626400948, -0.021726373583078384,...",0.769995


In [121]:
#integrate search and basic query
def ask(query):
  res = search_docs(query)
  dataSource = res.dataSource.iloc[0]
  #print(dataSource)
  response = get_response(query , dataSource)
  return response


In [122]:
#test for integration
result = ask("How is the course assess?")

In [123]:
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
result


Fix Cost (Tokens): 2054
Variable Cost (Tokens): 147


"Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.\n\nWhich session are you in (T01-T07)?"

#Input Aera

In [124]:
user_input=""
result = ask("Hi")
print(f'Chatbot said: \n{result}')
while (user_input!="stop"):
  user_input = input('Enter query: ')
  print(f'You entered: \n{user_input}')
  if user_input!="stop":
    result = ask(user_input)
    print(f'Chatbot said: {result}')

  print(f"Fix Cost (Tokens): "+str(fixed_tokens))
  print(f"Variable Cost (Tokens): "+str(variable_tokens))

Chatbot said: 
Hello! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.

Which session are you in (T01-T07)?
Enter query: T01
You entered: 
T01
Chatbot said: Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.

Which session are you in (T01-T07)?
Fix Cost (Tokens): 2054
Variable Cost (Tokens): 280
Enter query: stop
You entered: 
stop
Fix Cost (Tokens): 2054
Variable Cost (Tokens): 280


In [125]:
#anvil.server.wait_forever()